In [0]:
# Camada Bronze - Sets - Magic: The Gathering 
# Objetivo: Processo EL (Extract & Load) com nomenclatura padronizada
# Características: Extract da staging (S3/Parquet) -> Load na bronze (Unity Catalog/Delta) 
# Melhorias: Nomenclatura padronizada, colunas padronizadas, merge incremental robusto
# Fonte: MTG API (dados oficiais do Magic: The Gathering)

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import *
from delta.tables import DeltaTable

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

CATALOG_NAME = get_secret("catalog_name")
SCHEMA_NAME = "bronze"
TABLE_NAME = "TB_BRONZE_SETS"

S3_BUCKET = get_secret("s3_bucket")
S3_STAGE_PREFIX = get_secret("s3_stage_prefix", "magic_the_gathering/stage")
S3_BRONZE_PREFIX = get_secret("s3_bronze_prefix", "magic_the_gathering/bronze")
S3_STAGE_PATH = f"s3://{S3_BUCKET}/{S3_STAGE_PREFIX}"
S3_BRONZE_PATH = f"{S3_BUCKET}/{S3_BRONZE_PREFIX}"

# =============================================================================
# FUNÇÃO PARA CALCULAR PERÍODO TEMPORAL
# =============================================================================
def calculate_temporal_period(years_back=5):
    # Calcula período de 5 anos completos baseado no ano atual
    current_year = datetime.now().year
    start_year = current_year - years_back
    end_year = current_year  # Incluir o ano atual
    
    cutoff_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    
    return {
        'start_year': start_year,
        'end_year': end_year,
        'cutoff_date': cutoff_date,
        'end_date': end_date,
        'cutoff_date_str': cutoff_date.strftime("%Y-%m-%d"),
        'end_date_str': end_date.strftime("%Y-%m-%d")
    }

# Configurar período temporal
YEARS_BACK = 5
temporal_config = calculate_temporal_period(YEARS_BACK)

START_YEAR = temporal_config['start_year']
END_YEAR = temporal_config['end_year']
CUTOFF_DATE = temporal_config['cutoff_date']
END_DATE = temporal_config['end_date']
CUTOFF_DATE_STR = temporal_config['cutoff_date_str']
END_DATE_STR = temporal_config['end_date_str']

print("Iniciando pipeline EL Unity com nomenclatura padronizada - TB_BRONZE_SETS")
print("Dados de referência MTG API - com filtro temporal baseado em releaseDate")
print(f"Filtro temporal: {YEARS_BACK} anos incluindo atual ({START_YEAR} a {END_YEAR})")
print(f"Período: {CUTOFF_DATE_STR} até {END_DATE_STR}")

# =============================================================================
# MAPEAMENTO DE COLUNAS GOV
# =============================================================================
# Mapeamento das colunas originais para nomenclatura GOV (alinhado com Silver)
COLUMN_MAPPING = {
    # Identificação
    'code': 'COD_SET',
    'name': 'NME_SET',
    
    # Metadados
    'block': 'NME_BLOCK',
    'blockCode': 'COD_BLOCK',
    'parent': 'COD_PARENT',
    'parentCode': 'COD_PARENT_CODE',
    
    # Informações de lançamento
    'releaseDate': 'DT_RELEASE',
    'digital': 'FLG_DIGITAL',
    'foilOnly': 'FLG_FOIL_ONLY',
    
    # Informações de booster
    'booster': 'DESC_BOOSTER',
    'boosterV3': 'DESC_BOOSTER_V3',
    
    # URLs e Links (referências externas)
    'scryfallUri': 'URL_SCRYFALL',
    'scryfallApiUri': 'URL_SCRYFALL_API',
    'tcgplayerId': 'ID_TCGPLAYER',
    'cardmarketId': 'ID_CARDMARKET',
    
    # Novas colunas identificadas
    'type': 'NME_TYPE',
    'border': 'NME_BORDER',
    'mkm_id': 'ID_MKM',
    'mkm_name': 'NME_MKM',
    'gathererCode': 'COD_GATHERER',
    'magicCardsInfoCode': 'COD_MAGIC_CARDS_INFO',
    'oldCode': 'COD_OLD',
    'onlineOnly': 'FLG_ONLINE_ONLY',
    
    # Colunas de booster individuais
    'booster_0': 'DESC_BOOSTER_0',
    'booster_1': 'DESC_BOOSTER_1',
    'booster_2': 'DESC_BOOSTER_2',
    'booster_3': 'DESC_BOOSTER_3',
    'booster_4': 'DESC_BOOSTER_4',
    'booster_5': 'DESC_BOOSTER_5',
    'booster_6': 'DESC_BOOSTER_6',
    'booster_7': 'DESC_BOOSTER_7',
    'booster_8': 'DESC_BOOSTER_8',
    'booster_9': 'DESC_BOOSTER_9',
    'booster_10': 'DESC_BOOSTER_10',
    'booster_11': 'DESC_BOOSTER_11',
    'booster_12': 'DESC_BOOSTER_12',
    'booster_13': 'DESC_BOOSTER_13',
    'booster_14': 'DESC_BOOSTER_14',
    'booster_15': 'DESC_BOOSTER_15',
    'booster_16': 'DESC_BOOSTER_16',
    'booster_17': 'DESC_BOOSTER_17',
    'booster_18': 'DESC_BOOSTER_18',
    'booster_19': 'DESC_BOOSTER_19',
    
    # Metadados
    'source': 'NME_SOURCE',
    'ingestion_timestamp': 'DT_INGESTION',
    
    # Datas
    'created_at': 'DT_CREATED',
    'updated_at': 'DT_UPDATED'
}


In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================
def setup_unity_catalog():
    # Configura o Unity Catalog e schema
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
        spark.sql(f"USE CATALOG {CATALOG_NAME}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
        spark.sql(f"USE SCHEMA {SCHEMA_NAME}")
        return True
    except Exception as e:
        print(f"Erro ao configurar Unity Catalog: {e}")
        return False

def extract_from_staging(table_name):
    # EXTRACT: Lê dados da camada staging
    try:
        print(f"Iniciando extract_from_staging para {table_name}")
        
        # Para TB_BRONZE_SETS, ler do arquivo sets da staging
        if table_name == "TB_BRONZE_SETS":
            stage_path = f"{S3_STAGE_PATH}/*_sets.parquet"
        else:
            stage_path = f"{S3_STAGE_PATH}/*_{table_name}.parquet"
        
        print(f"Tentando ler dados de: {stage_path}")
        
        df = spark.read.parquet(stage_path)
        count = df.count()
        print(f"Extraídos {count} registros de staging para {table_name}")
        
        if count == 0:
            print("AVISO: Nenhum registro encontrado em staging")
            return None
            
        return df
    except Exception as e:
        print(f"Erro no EXTRACT de staging: {e}")
        print(f"Path tentado: {stage_path}")
        return None

def apply_governance_naming(df):
    # Aplica nomenclatura GOV nas colunas
    try:
        available_columns = df.columns
        column_mapping = {}
        renamed_columns = []
        for original_col, gov_col in COLUMN_MAPPING.items():
            if original_col in available_columns:
                column_mapping[original_col] = gov_col
                renamed_columns.append((original_col, gov_col))
        for original_col, gov_col in column_mapping.items():
            df = df.withColumnRenamed(original_col, gov_col)
        if renamed_columns:
            print("Colunas renomeadas para governança:")
            for orig, gov in renamed_columns:
                print(f"  {orig} → {gov}")
        else:
            print("Nenhuma coluna renomeada para governança.")
        return df
    except Exception as e:
        print(f"Erro ao aplicar nomenclatura GOV: {e}")
        return df

def apply_temporal_filter(df, table_name):
    # Aplica filtro temporal de 5 anos
    if table_name == "TB_BRONZE_SETS":
        # Para TB_BRONZE_SETS (dados MTG API), aplicar filtro baseado no releaseDate
        # Filtro: sets lançados nos últimos 5 anos
        from pyspark.sql.functions import col
        
        try:
            total_before = df.count()
            df_filtered = df.filter(
                (col("releaseDate") >= CUTOFF_DATE_STR) & 
                (col("releaseDate") <= END_DATE_STR)
            )
            total_after = df_filtered.count()
            
            print(f"Filtro temporal aplicado para TB_BRONZE_SETS:")
            print(f"  - Período: {CUTOFF_DATE_STR} até {END_DATE_STR}")
            print(f"  - Anos incluindo atual: {START_YEAR} a {END_YEAR}")
            print(f"  - Registros antes: {total_before}")
            print(f"  - Registros depois: {total_after}")
            print(f"  - Registros removidos: {total_before - total_after}")
            
            return df_filtered
            
        except Exception as e:
            print(f"Erro ao aplicar filtro temporal: {e}")
            print("Continuando sem filtro temporal...")
            return df
    
    return df

def check_delta_table_exists(delta_path):
    # Verifica se a tabela Delta existe
    try:
        # Verificar se _delta_log existe
        delta_log_path = f"{delta_path}/_delta_log"
        try:
            files = dbutils.fs.ls(delta_log_path)
            if files:
                # Tentar ler a tabela para verificar se tem dados
                df = spark.read.format("delta").load(delta_path)
                count = df.count()
                print(f"Tabela Delta existe com {count} registros")
                return True, count > 0
            else:
                return False, False
        except:
            return False, False
    except Exception as e:
        print(f"Erro ao verificar tabela Delta: {e}")
        return False, False

def get_delta_schema_for_merge(delta_path):
    # Obtém o schema da tabela Delta existente, excluindo colunas de particionamento
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        
        # Excluir colunas de particionamento do schema para merge
        partition_columns = ['RELEASE_YEAR', 'RELEASE_MONTH']
        merge_schema = [col for col in schema_fields if col not in partition_columns]
        
        print(f"Schema completo da tabela Delta: {schema_fields}")
        print(f"Schema para merge (sem particionamento): {merge_schema}")
        
        # Garantir que todas as colunas necessárias estejam presentes
        expected_columns = ['COD_SET', 'NME_SET', 'DT_RELEASE', 'NME_SOURCE']
        missing_columns = [col for col in expected_columns if col not in merge_schema]
        
        if missing_columns:
            print(f"AVISO: Colunas faltando no schema para merge: {missing_columns}")
            # Adicionar colunas faltantes ao schema
            merge_schema.extend(missing_columns)
            print(f"Schema atualizado para merge: {merge_schema}")
        
        return merge_schema
    except Exception as e:
        print(f"Erro ao obter schema Delta: {e}")
        return []

def prepare_dataframe_for_merge(df, table_name, target_schema=None):
    # Prepara DataFrame para merge, garantindo compatibilidade de schema
    if not df:
        return None
    try:
        # Aplica nomenclatura GOV
        df = apply_governance_naming(df)

        # Lista de colunas padronizadas (GOV) para sets
        colunas_padrao = [
            'COD_SET', 'NME_SET', 'NME_TYPE', 'NME_BORDER', 'ID_MKM', 'NME_MKM', 'DT_RELEASE',
            'COD_GATHERER', 'COD_MAGIC_CARDS_INFO', 'DESC_BOOSTER', 'COD_OLD', 'FLG_ONLINE_ONLY',
            'NME_SOURCE',
            'DESC_BOOSTER_0', 'DESC_BOOSTER_1', 'DESC_BOOSTER_2', 'DESC_BOOSTER_3',
            'DESC_BOOSTER_4', 'DESC_BOOSTER_5', 'DESC_BOOSTER_6',
            'DESC_BOOSTER_7', 'DESC_BOOSTER_8', 'DESC_BOOSTER_9', 'DESC_BOOSTER_10',
            'DESC_BOOSTER_11', 'DESC_BOOSTER_12', 'DESC_BOOSTER_13', 'DESC_BOOSTER_14',
            'DESC_BOOSTER_15', 'DESC_BOOSTER_16', 'DESC_BOOSTER_17', 'DESC_BOOSTER_18', 'DESC_BOOSTER_19',
            'RELEASE_YEAR', 'RELEASE_MONTH',
            'FLG_DIGITAL', 'FLG_FOIL_ONLY', 'COD_BLOCK', 'COD_PARENT', 'COD_PARENT_CODE',
            'URL_SCRYFALL', 'URL_SCRYFALL_API', 'ID_TCGPLAYER', 'ID_CARDMARKET',
            'DT_INGESTION', 'DT_CREATED', 'DT_UPDATED'
        ]

        # Remover colunas não padronizadas
        for col in df.columns:
            if col not in colunas_padrao:
                df = df.drop(col)
                print(f"Coluna '{col}' removida do DataFrame (não está na governança)")

        # Adicionar colunas de particionamento se não existirem
        from pyspark.sql.functions import year, month, to_date, col as pyspark_col
        if 'DT_RELEASE' in df.columns:
            df = df.withColumn('DT_RELEASE', to_date(pyspark_col('DT_RELEASE')))
            df = df.withColumn('RELEASE_YEAR', year(pyspark_col('DT_RELEASE')))
            df = df.withColumn('RELEASE_MONTH', month(pyspark_col('DT_RELEASE')))

        # Se temos schema de destino, filtrar colunas compatíveis
        if target_schema:
            available_columns = df.columns
            compatible_columns = [col for col in available_columns if col in target_schema]
            if not compatible_columns:
                print("Nenhuma coluna compatível encontrada para merge")
                return None
            print(f"Colunas compatíveis para merge: {compatible_columns}")
            df = df.select(compatible_columns)

        return df
    except Exception as e:
        print(f"Erro ao preparar DataFrame para merge: {e}")
        return None

def execute_merge_with_specific_columns(delta_table, merge_df, merge_columns):
    # Executa merge especificando exatamente quais colunas atualizar
    try:
        # Construir a condição de merge (sets usa 'COD_SET' como chave)
        merge_condition = "bronze.COD_SET = novo.COD_SET"
        
        # Verificar se as colunas existem no DataFrame de merge
        merge_df_columns = merge_df.columns
        print(f"Colunas disponíveis no DataFrame de merge: {merge_df_columns}")
        
        # Construir as ações de UPDATE especificando colunas
        update_actions = {}
        for col_name in merge_columns:
            if col_name != 'COD_SET' and col_name in merge_df_columns:  # Não atualizar a chave de merge
                update_actions[col_name] = f"novo.{col_name}"
        
        # Construir as ações de INSERT especificando colunas
        insert_actions = {}
        for col_name in merge_columns:
            if col_name in merge_df_columns:
                insert_actions[col_name] = f"novo.{col_name}"
        
        print(f"Executando merge com {len(update_actions)} colunas para UPDATE")
        print(f"Colunas para UPDATE: {list(update_actions.keys())}")
        print(f"Colunas para INSERT: {list(insert_actions.keys())}")
        
        # Executar merge com ações específicas
        delta_table.alias("bronze").merge(
            merge_df.alias("novo"),
            merge_condition
        ).whenMatchedUpdate(set=update_actions) \
         .whenNotMatchedInsert(values=insert_actions) \
         .execute()
        
        print("Merge Delta executado com sucesso")
        return True
    except Exception as e:
        print(f"Erro no merge Delta: {e}")
        return False

def check_unity_table_exists(full_table_name):
    # Verifica se a tabela Unity Catalog existe
    try:
        # Tentar fazer uma consulta simples na tabela
        test_query = f"SELECT 1 FROM {full_table_name} LIMIT 1"
        spark.sql(test_query)
        print(f"Tabela Unity Catalog '{full_table_name}' existe")
        return True
    except Exception as e:
        print(f"Tabela Unity Catalog '{full_table_name}' não existe ou não está acessível")
        return False

def create_or_update_unity_catalog(full_table_name, delta_path):
    # Cria ou atualiza tabela Unity Catalog preservando modificações existentes
    try:
        table_exists = check_unity_table_exists(full_table_name)
        
        if not table_exists:
            # Criar tabela pela primeira vez
            print(f"Criando tabela Unity Catalog: {full_table_name}")
            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{delta_path}'
            COMMENT 'Tabela bronze de referência de sets do Magic: The Gathering com nomenclatura padronizada'
            """
            spark.sql(create_table_sql)
            print(f"Tabela Unity Catalog criada: {full_table_name}")
        else:
            # Tabela já existe, apenas atualizar propriedades de pipeline (sem perder modificações)
            print(f"Tabela Unity Catalog já existe: {full_table_name}")
            print("Atualizando apenas propriedades de pipeline (preservando modificações existentes)")
        
        # Sempre atualizar propriedades de pipeline (não afeta modificações customizadas)
        try:
            spark.sql(f"""
            ALTER TABLE {full_table_name} SET TBLPROPERTIES (
                'bronze_layer' = 'true',
                'data_source' = 'mtg_api',
                'last_processing_date' = '{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
                'governance_applied' = 'true',
                'nomenclature_standard' = 'GOV',
                'merge_strategy' = 'incremental',
                'reference_data' = 'true',
                'temporal_filter' = 'releaseDate',
                'temporal_period' = '{START_YEAR}_to_{END_YEAR}'
            )
            """)
            print("Propriedades de pipeline atualizadas")
        except Exception as e:
            print(f"Erro ao atualizar propriedades: {e}")
        
        return True
    except Exception as e:
        print(f"Erro ao criar/atualizar Unity Catalog: {e}")
        return False

def load_to_bronze_unity_incremental_gov(df, table_name):
    # LOAD: Carrega dados na camada bronze (Unity + Incremental + GOV)
    if not df:
        return None
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        
        # Importar DeltaTable aqui para garantir que está disponível
        try:
            from delta.tables import DeltaTable
            print("DeltaTable importado com sucesso")
        except ImportError as e:
            print(f"Erro ao importar DeltaTable: {e}")
            return None
        
        print(f"Carregando dados para bronze: {table_name}")
        print(f"Unity Catalog: {full_table_name}")
        print(f"Delta Path: {delta_path}")
        
        # Verificar se tabela Delta existe
        delta_exists, has_data = check_delta_table_exists(delta_path)
        
        if not delta_exists or not has_data:
            # Primeira criação com particionamento
            print("Criando tabela Delta pela primeira vez com nomenclatura GOV...")
            df_processed = prepare_dataframe_for_merge(df, table_name)
            
            if table_name == "TB_BRONZE_SETS":
                # Para TB_BRONZE_SETS, particionamento por ano de lançamento
                from pyspark.sql.functions import year, month
                
                # Criar colunas de particionamento baseadas no releaseDate
                df_with_partition = df_processed.withColumn("RELEASE_YEAR", 
                                  year(col("DT_RELEASE"))) \
                       .withColumn("RELEASE_MONTH", 
                                  month(col("DT_RELEASE")))
                
                print(f"Particionamento criado com releaseDate real dos sets")
                
                df_with_partition.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .partitionBy("RELEASE_YEAR", "RELEASE_MONTH") \
                       .save(delta_path)
            else:
                df_processed.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .save(delta_path)
            print("Tabela Delta criada com sucesso")
        else:
            # Verificar se o schema está correto
            print("Tabela Delta existe, verificando schema...")
            schema_ok = check_and_fix_delta_schema(delta_path, table_name)
            
            if not schema_ok:
                # Schema incorreto, recriar tabela
                print("Recriando tabela Delta com schema correto...")
                df_processed = prepare_dataframe_for_merge(df, table_name)
                
                if table_name == "TB_BRONZE_SETS":
                    # Para TB_BRONZE_SETS, particionamento por ano de lançamento
                    from pyspark.sql.functions import year, month
                    
                    # Criar colunas de particionamento baseadas no releaseDate
                    df_with_partition = df_processed.withColumn("RELEASE_YEAR", 
                                      year(col("DT_RELEASE"))) \
                           .withColumn("RELEASE_MONTH", 
                                      month(col("DT_RELEASE")))
                    
                    print(f"Particionamento criado com releaseDate real dos sets")
                    
                    df_with_partition.write.format("delta") \
                           .mode("overwrite") \
                           .option("overwriteSchema", "true") \
                           .partitionBy("RELEASE_YEAR", "RELEASE_MONTH") \
                           .save(delta_path)
                else:
                    df_processed.write.format("delta") \
                           .mode("overwrite") \
                           .option("overwriteSchema", "true") \
                           .save(delta_path)
                print("Tabela Delta recriada com sucesso")
            else:
                # Schema correto, fazer merge incremental
                print("Schema correto, preparando para merge incremental...")
                target_schema = get_delta_schema_for_merge(delta_path)
                
                # Preparar DataFrame para merge
                merge_df = prepare_dataframe_for_merge(df, table_name, target_schema)
                if not merge_df:
                    print("Nenhum DataFrame compatível para merge")
                    return None
                
                # Executar merge com colunas específicas (sem particionamento)
                print("Executando merge Delta...")
                try:
                    delta_table = DeltaTable.forPath(spark, delta_path)
                    print("DeltaTable criado com sucesso")
                except Exception as e:
                    print(f"Erro ao criar DeltaTable: {e}")
                    return None
                
                # Usar função específica para merge com colunas definidas
                merge_success = execute_merge_with_specific_columns(delta_table, merge_df, target_schema)
                if not merge_success:
                    print("Falha no merge Delta")
                    return None

        # Criar ou atualizar tabela Unity Catalog (PRESERVANDO MODIFICAÇÕES)
        create_or_update_unity_catalog(full_table_name, delta_path)
        
        return df
    except Exception as e:
        print(f"Erro no LOAD para bronze {table_name}: {e}")
        return None

def process_el_unity_incremental_gov(table_name):
    # Executa o pipeline EL Unity + Incremental + GOV
    print(f"Iniciando process_el_unity_incremental_gov para {table_name}")
    
    stage_df = extract_from_staging(table_name)
    print(f"Resultado do extract_from_staging: {stage_df is not None}")
    
    if not stage_df:
        print(f"Falha no EXTRACT de staging para {table_name}")
        return None
    
    print(f"DataFrame de staging obtido com {stage_df.count()} registros")
    
    result_df = load_to_bronze_unity_incremental_gov(stage_df, table_name)
    print(f"Resultado do load_to_bronze_unity_incremental_gov: {result_df is not None}")
    
    return result_df

def query_bronze_unity(table_name):
    # Consulta na tabela bronze Unity Catalog
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        count_query = f"SELECT COUNT(*) as total FROM {full_table_name}"
        count_result = spark.sql(count_query)
        count_result.show()
        
    except Exception as e:
        print(f"Erro ao consultar tabela bronze: {e}")

def show_delta_info(table_name):
    # Mostra informações da tabela Delta
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        from delta.tables import DeltaTable
        delta_table = DeltaTable.forPath(spark, delta_path)
        history = delta_table.history()
        print(f"Versões Delta: {history.count()}")
            
    except Exception as e:
        print(f"Erro ao mostrar informações Delta: {e}")

def show_sample_data(table_name):
    # Mostra dados de exemplo da tabela
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        sample_query = f"SELECT * FROM {full_table_name} LIMIT 5"
        print("Dados de exemplo:")
        spark.sql(sample_query).show(truncate=False)
        
    except Exception as e:
        print(f"Erro ao mostrar dados de exemplo: {e}")

def show_governance_info():
    # Mostra informações sobre a governança aplicada
    print("=" * 60)
    print("INFORMAÇÕES DE GOVERNAÇA - TB_BRONZE_SETS")
    print("=" * 60)
    print(f"Nomenclatura GOV aplicada: {len(COLUMN_MAPPING)} colunas")
    print(f"Período temporal: {YEARS_BACK} anos incluindo atual ({START_YEAR} a {END_YEAR})")
    print(f"Filtro baseado em: releaseDate dos sets")
    print(f"Particionamento: RELEASE_YEAR e RELEASE_MONTH")
    print(f"Chave de merge: COD_SET")
    
    print("\n=== MAPEAMENTO PRINCIPAL DE COLUNAS ===")
    print("Identificação:")
    print(f"  code → COD_SET")
    print(f"  name → NME_SET")
    
    print("\nMetadados:")
    print(f"  type → NME_TYPE")
    print(f"  border → NME_BORDER")
    print(f"  block → NME_BLOCK")
    print(f"  blockCode → COD_BLOCK")
    
    print("\nIdentificadores:")
    print(f"  mkm_id → ID_MKM")
    print(f"  mkm_name → NME_MKM")
    print(f"  gathererCode → COD_GATHERER")
    print(f"  magicCardsInfoCode → COD_MAGIC_CARDS_INFO")
    print(f"  oldCode → COD_OLD")
    
    print("\nFlags:")
    print(f"  digital → FLG_DIGITAL")
    print(f"  foilOnly → FLG_FOIL_ONLY")
    print(f"  onlineOnly → FLG_ONLINE_ONLY")
    
    print("\nDatas:")
    print(f"  releaseDate → DT_RELEASE")
    print(f"  created_at → DT_CREATED")
    print(f"  updated_at → DT_UPDATED")
    print(f"  ingestion_timestamp → DT_INGESTION")
    
    print("\nBooster:")
    print(f"  booster → DESC_BOOSTER")
    print(f"  booster_0 a booster_6 → DESC_BOOSTER_0 a DESC_BOOSTER_6")
    
    print("\nURLs (referências externas):")
    print(f"  scryfallUri → URL_SCRYFALL")
    print(f"  scryfallApiUri → URL_SCRYFALL_API")
    
    print("\nIDs:")
    print(f"  tcgplayerId → ID_TCGPLAYER")
    print(f"  cardmarketId → ID_CARDMARKET")
    
    print("=" * 60)

def check_and_fix_delta_schema(delta_path, table_name):
    # Verifica se o schema da tabela Delta está correto, se não estiver, deleta e recria
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        
        print(f"Schema atual da tabela Delta: {schema_fields}")
        
        # Definir schema esperado para cada tabela
        if table_name == "TB_BRONZE_FORMATS":
            expected_schema = ['NME_FORMAT', 'DT_INGESTION', 'NME_SOURCE', 'NME_ENDPOINT', 'INGESTION_YEAR', 'INGESTION_MONTH']
        elif table_name == "TB_BRONZE_SETS":
            expected_schema = ['COD_SET', 'NME_SET', 'DT_RELEASE', 'NME_SOURCE', 'RELEASE_YEAR', 'RELEASE_MONTH']
        else:
            # Para outras tabelas, não verificar schema específico
            return True
        
        # Verificar se todas as colunas esperadas estão presentes
        missing_columns = [col for col in expected_schema if col not in schema_fields]
        incorrect_columns = [col for col in schema_fields if col in ['endpoint', 'source', 'ingestion_timestamp']]  # Colunas antigas
        
        if missing_columns or incorrect_columns:
            print(f"Schema incorreto detectado:")
            if missing_columns:
                print(f"  - Colunas faltando: {missing_columns}")
            if incorrect_columns:
                print(f"  - Colunas com nomenclatura incorreta: {incorrect_columns}")
            
            print("Deletando tabela Delta para recriação...")
            
            # Deletar tabela Delta
            try:
                dbutils.fs.rm(delta_path, recurse=True)
                print(f"Tabela Delta deletada: {delta_path}")
                return False  # Indica que precisa recriar
            except Exception as e:
                print(f"Erro ao deletar tabela Delta: {e}")
                return False
        else:
            print("✅ Schema correto - Tabela não precisa ser recriada")
            return True
            
    except Exception as e:
        print(f"Erro ao verificar schema Delta: {e}")
        return False

In [0]:

# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================

try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()

setup_success = setup_unity_catalog()
if not setup_success:
    raise Exception("Falha ao configurar Unity Catalog")

# Executar pipeline com tratamento de erro robusto
try:
    sets_bronze_df = process_el_unity_incremental_gov("TB_BRONZE_SETS")
    
    if sets_bronze_df:
        print("Pipeline executado com sucesso")
        
        # Verificações e informações
        query_bronze_unity("TB_BRONZE_SETS")
        show_delta_info("TB_BRONZE_SETS")
        #show_sample_data("TB_BRONZE_SETS")
        show_governance_info()
    else:
        print("Falha no pipeline - DataFrame retornado é None")
        
except Exception as e:
    print(f"Erro durante execução do pipeline: {e}")
    print("Falha no pipeline") 